# Machine learning - LightGBM

In [1]:
import os
import pandas as pd

i=0
coins=[]
for file in [x for x in os.listdir("data/wrangled/") if "csv" in x]:
    print ("loading "+file)
    temp_df=pd.read_csv("data/wrangled/"+file)
    temp_df["coin"]=i 
    coins.append(file[:-13])
    if i:
        full_df=full_df.append(temp_df)
    else:
        full_df=temp_df
    i+=1

loading Stellar_wrangled.csv
loading Polkadot_wrangled.csv
loading Tron_wrangled.csv
loading Litecoin_wrangled.csv
loading WrappedBitcoin_wrangled.csv
loading ChainLink_wrangled.csv
loading Cardano_wrangled.csv
loading BinanceCoin_wrangled.csv
loading Tether_wrangled.csv
loading Aave_wrangled.csv
loading CryptocomCoin_wrangled.csv
loading Cosmos_wrangled.csv
loading Monero_wrangled.csv
loading NEM_wrangled.csv
loading XRP_wrangled.csv
loading Bitcoin_wrangled.csv
loading Ethereum_wrangled.csv
loading Solana_wrangled.csv
loading USDCoin_wrangled.csv
loading Dogecoin_wrangled.csv
loading Uniswap_wrangled.csv
loading Iota_wrangled.csv
loading EOS_wrangled.csv


In [25]:
full_df.sample()

,Performance,log_and_diffed_High,log_and_diffed_Low,log_and_diffed_Open,log_and_diffed_Close,log_and_diffed_Volume,log_and_diffed_Marketcap,coin
1620,1.771252,-0.014365,-0.010361,-0.029119,0.01575,-0.006315,0.015879,15


In [4]:
full_df.columns

Index(['Performance', 'log_and_diffed_High', 'log_and_diffed_Low',
       'log_and_diffed_Open', 'log_and_diffed_Close', 'log_and_diffed_Volume',
       'log_and_diffed_Marketcap', 'coin'],
      dtype='object')

In [5]:
len(full_df)

36079

In [3]:
full_df.reset_index(drop=True,inplace=True)

In [2]:
import numpy as np
full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
full_df.dropna(inplace=True)
len(full_df)

36077

**1st attempt - 80-20 train/test split**

In [7]:
0.2-36079
#7215 results for testing

7215.8

In [5]:
import random

window=40

test_sample=random.sample(range(window,len(full_df)),k=7215)

y_test=full_df.iloc[test_sample,:]
X_test=[]

from tqdm import tqdm 

for i in tqdm(test_sample):
    temp1=[]
    for x in range(i-window,i):
        temp1.append(list(full_df.iloc[x,:]))
    X_test.append([item for sublist in temp1 for item in sublist])

100%|██████████| 7215/7215 [00:49<00:00, 144.58it/s]


In [7]:
X=full_df.drop(labels=test_sample,axis=0)
X_train=[]
Y_train=[]

from tqdm import tqdm 

for i in tqdm(range(window,len(full_df))):
    temp2=[]
    for x in range(i-window,i):
        temp2.append(list(full_df.iloc[x,:]))
    X_train.append([item for sublist in temp2 for item in sublist])
    Y_train.append(list(full_df.iloc[i,:]))

100%|██████████| 36037/36037 [04:00<00:00, 150.03it/s]


In [8]:
Y_train_df=pd.DataFrame(Y_train)

In [9]:
from lightgbm import LGBMRegressor

model= LGBMRegressor(boosting='dart',n_estimators=1000,max_depth=150,num_leaves=80,n_jobs=-1)

y_pred=[]
for i in tqdm(range(len(Y_train_df.columns))):
    model.fit(X=X_train, y=Y_train_df.iloc[:,i])
    y_pred.append(model.predict(X_test))

  0%|          | 0/8 [00:00<?, ?it/s]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 12%|█▎        | 1/8 [06:03<42:26, 363.73s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 25%|██▌       | 2/8 [11:44<34:59, 349.94s/it]

...

 75%|███████▌  | 6/8 [35:13<11:26, 343.32s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 88%|████████▊ | 7/8 [40:45<05:39, 339.63s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


100%|██████████| 8/8 [45:28<00:00, 341.06s/it]


In [10]:
from sklearn.metrics import mean_squared_error

y_test_df=pd.DataFrame(y_test)

for i in range(len(y_pred)):
    print(mean_squared_error(y_test_df.iloc[:,i],y_pred[i],squared=False))

2.5080562162709352
0.018498463968929926
0.01604757835944891
0.006080501587157533
0.023902443775367654
0.18087269077170284
0.024959497899357617
0.1250543435280786


In [16]:
y_test_df.iloc[5,:]

Performance                -5.523362
log_and_diffed_High        -0.009984
log_and_diffed_Low          0.125268
log_and_diffed_Open         0.216190
log_and_diffed_Close       -0.062195
log_and_diffed_Volume      -2.096137
log_and_diffed_Marketcap   -0.062195
coin                        0.000000
Name: 554, dtype: float64

In [15]:
y_pred_df=pd.DataFrame(y_pred)
y_pred_df.iloc[:,5]

0   -3.980894
1   -0.001676
2    0.128343
3    0.211054
4   -0.041001
5   -1.735502
6   -0.049900
7    0.017270
Name: 5, dtype: float64

In [ ]:
model.save_model('model1.txt')
# model = lgb.Booster(model_file='lgb_classifier.txt')

Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=10, verbose=2)
np.mean(-scores)